In [50]:
import pandas as pd
import os
import numpy as np

csv_files = [f for f in os.listdir('../data/raw') if f.endswith('.csv')]

In [61]:
df_dirty = pd.read_csv(f'../data/raw/{csv_files[0]}')
df_dirty.columns

Index(['engine', 'se_link', 'se_title', 'se_description', 'se_source',
       'n3k_link', 'n3k_title', 'n3k_body', 'n3k_author', 'n3k_published',
       'bs_link', 'bs_title', 'bs_body'],
      dtype='object')

In [62]:
# displaying all three links (se_link, n3k_link, bs_link) if they are not identical
df_dirty.loc[(df_dirty['se_link'] != df_dirty['n3k_link']) | (df_dirty['se_link'] != df_dirty['bs_link'])]

# Links are always identical here, so we can drop two of them and rename the remaining one
df_dirty.drop(['n3k_link', 'bs_link'], axis=1, inplace=True)
df_dirty.rename(columns={'se_link': 'link'}, inplace=True)

In [72]:
# displaying the three titles if they are not identical
showme = df_dirty.loc[(df_dirty['se_title'] != df_dirty['n3k_title']) 
                      | (df_dirty['se_title'] != df_dirty['bs_title'])][['se_title', 'n3k_title', 'bs_title']]

In [54]:
# Removing all '\n' and '\t' from n3k_title, n3k_body, bs_title, bs_body as well as leading and trailing whitespaces and more than
# one whitespace in between words and replacing them with np.nan if they are empty strings
for col in ['n3k_title', 'n3k_body', 'bs_title', 'bs_body']:
    df_dirty[col] = df_dirty[col].str.replace('\n', ' ').str.replace('\t', ' ').str.replace(' +', ' ').str.strip()
    df_dirty.loc[df_dirty[col] == '', col] = np.nan    

# replacing n3k_titles with NA if they contain certain words and are less than 20 characters
for word in ['robot', 'subscribe', 'register']:
    df_dirty.loc[(df_dirty['n3k_title'].str.lower().str.contains(word))
                 & (df_dirty['n3k_title'].str.len() < 20), 'n3k_title'] = np.nan
    
# doing the same with n3k_body
for word in ['cookies', 'javascript', 'register', 'explorer', 'benzinga']:
    df_dirty.loc[(df_dirty['n3k_body'].str.lower().str.contains(word))
                 & (df_dirty['n3k_body'].str.len() < 400), 'n3k_body'] = np.nan

# Doing the same for bs_title and bs_body
for word in ['yahoo finance', 'bloomberg', 'yahoo news', 'navigation', 'the straits times']:
    df_dirty.loc[(df_dirty['bs_title'].str.lower().str.contains(word))
                 & (df_dirty['bs_title'].str.len() < 20), 'bs_title'] = np.nan

for word in ['javascript', 'copyright', 'benzinga']:
    df_dirty.loc[(df_dirty['bs_body'].str.lower().str.contains(word))
                 & (df_dirty['bs_body'].str.len() < 400), 'bs_body'] = np.nan

/var/folders/h4/tbxkzqv53pd3f16_r3jbmshw0000gn/T/ipykernel_5381/1064490035.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dirty[col] = df_dirty[col].str.replace('\n', ' ').str.replace('\t', ' ').str.replace(' +', ' ').str.strip()


In [ ]:
# Deciding weather to keep bs_title or n3k_title and bs_body or n3k_body
# Does it make sense to keep one part from one library and another part from another library knowing that its the same article?

# Keeping both title and body of the library that has a longer body
# E.g.: If for article 1, the n3k body is longer, we keep both  n3k_title and n3k_body

for row in df_dirty:
    if df_dirty['n3k_body'].str.len() > df_dirty['bs_body'].str.len():
        df_dirty['title'] = df_dirty['n3k_title']
        df_dirty['body'] = df_dirty['n3k_body']
    else:
        df_dirty['title'] = df_dirty['bs_title']
        df_dirty['body'] = df_dirty['bs_body']

df_dirty.drop(['n3k_title', 'n3k_body', 'bs_title', 'bs_body'], axis=1, inplace=True)

In [ ]:
# Creating a loop that applies the above rules to each table in 'data/raw' and saves the cleaned table to 
# 'data/interim'
for csv_file in csv_files:
    df_dirty = pd.read_csv(f'../data/raw/{csv_file}')
